In [1]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import config
import time


In [56]:
#specify database for connection
db = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "Movies")

cursor = db.cursor()

In [3]:
#create database
from mysql.connector import errorcode
db_name = 'Movies'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [3]:
import requests
from bs4 import BeautifulSoup as BS

In [5]:
#first page

page1 = requests.get('https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start=251&ref_=adv_nxt')
soup1 = BS(page1.content, 'html.parser')
soup1.prettify()
info1 = soup1.find_all(id = 'main')
movie1 = info1[0].find_all(class_='lister-item mode-advanced')

In [6]:
# insert_movies(make_df(movie1))

In [7]:
def get_score(movie):
        metascores = []
        for score in movie:
            metascore1 = score.find(class_="metascore")
            if metascore1 == None:
                metascore = 'Null'
                metascores.append(metascore)
            else:
                metascores.append(int(metascore1.get_text(strip=True)))
        return metascores
# len(get_score(movie))

In [8]:
def find_fk(movie):
    ids = []
    for m in movie:
        id = m.find (class_="starBarWidget")['id']
        ids.append(id)
    return ids

# find_fk(movie1)

In [9]:
def get_titles(movie):

    titles = []
    for one in movie:
        title = one.find('img')['alt']
        titles.append(title) 
    return titles

# len(get_titles(movie))

In [10]:
def get_stars(movie):
    stars = []
    for one in movie:
        try:
            stars.append(float(one.find(class_='inline-block ratings-imdb-rating')['data-value']))
        except:
            stars.append('Null')
#         stars.append(float(x['data-value']))
    return stars

# len(get_stars(movie))

In [11]:
def get_year(movie):
    
    years = []
    for one in movie:
        try:
            year = one.find(class_="lister-item-year text-muted unbold").get_text()
            parsed_year = year[-5:-1]
            years.append(int(parsed_year))
        except:
            years.append(2019)
    return years
# get_year(movie1)

In [12]:
def get_genre(movie):
    genres = []
    for m in movie:
        genre = m.find(class_='genre')
        try:
            genre1 = str.split(genre.get_text(strip=True), sep = ',')[0]
        except:
            genre1 = genre
        genres.append(genre1)
    return genres
# len(get_genre(movie1))

In [13]:
def get_gross(movie):
    gross_list = []
    
    for m in movie:
        gross = m.find(class_='sort-num_votes-visible')
        parsed_gross = gross.get_text(strip=True)
        parsed_gross2 = str.split(parsed_gross, sep='|')
       # print(parsed_gross2[1])
        if 'Gross' in parsed_gross2[0]:
            parsed_gross2 = parsed_gross2[0]
#             print(parsed_gross2[0])
            parsed_gross3 = str.split(parsed_gross2, sep=':')
#             print(parsed_gross3)
            parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
            final_gross = str.split(parsed_gross4, sep='$')[1]
            gross_list.append(int(final_gross))
#             print(parsed_gross2)
        else:
            try:       
                parsed_gross3 = str.split(parsed_gross2[1], sep=':')
                parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
                final_gross = str.split(parsed_gross4, sep='$')[1]
                gross_list.append(int(final_gross))
            except: 
                gross_list.append(0)
    
    return gross_list

# len(get_gross(movie))

In [41]:
#create table
DB_NAME = 'Movies'

create_query = """
      CREATE TABLE imdb_movies
    (
      imdb_id varchar(24),
      title varchar(50),
      year INT(4),
      star_rating FLOAT(3),
      metascore INT(5),
      gross INT(5),
      genre varchar(30),
      CONSTRAINT primary_key PRIMARY KEY(imdb_id)
      )
    """



In [ ]:
#make into DF
def make_df(movie):
    all_movies_dataframe = pd.DataFrame({
        "imdb_id": find_fk(movie),
        "title": get_titles(movie),
        "year": get_year(movie),
        "star_rating": get_stars(movie),
        "metascore": get_score(movie),
        "gross": get_gross(movie),
        "genre": get_genre(movie)
    })
    return all_movies_dataframe


In [ ]:
#uses DF to insert into mySQL
def insert_movies(dataframe):
    all_movies_tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO imdb_movies (imdb_id, title, year, star_rating, metascore, gross, genre) 
            VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    for movie in all_movies_tuples:
        
        cursor.execute(insert_statement, movie)
        print(movie)
    
#     cursor.executemany(insert_statement, all_movies_tuples)
    db.commit()


In [ ]:
var = 251
while var < 750: 
    page = requests.get(f"https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start={var}&ref_=adv_nxt")
    soup = BS(page.content, 'html.parser')
    soup.prettify()
    info = soup.find_all(id = 'main')
    movie = info[0].find_all(class_='lister-item mode-advanced')
    movie_df = make_df(movie)
    insert_movies(movie_df)
    # #     print(movie_df)
    var +=50


In [51]:
cursor = db.cursor()

df = cursor.execute("""SELECT imdb_id FROM Movies.imdb_movies2;""")

df = pd.DataFrame(cursor.fetchall())

df.columns = [x[0] for x in cursor.description]

list_of_ids = df.values.tolist()

list_of_imdb_ids = [val[0] for val in list_of_ids]

# list_of_imdb_ids

In [5]:
api_key = '0bf0d1d80e2ab08ed768c022e7f27155'
all_info = []
for id_ in list_of_imdb_ids:
    find_external_id = requests.get(f'https://api.themoviedb.org/3/find/{id_}?api_key=0bf0d1d80e2ab08ed768c022e7f27155&language=en-US&external_source=imdb_id').json()
    find_external_id['imdb_id']=id_
    all_info.append(find_external_id)
    print(all_info[:5])

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}]
[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: B

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

...

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

In [71]:
def make_df(all_info):
    mymoviedatabase_df = pd.DataFrame({
        "imdb_id": list_of_imdb_ids,
        "title": get_title(all_info),
        "release_date": get_release_date(all_info),
        "overview": get_overview(all_info),
        })
    return mymoviedatabase_df

make_df(all_info).head()

,imdb_id,title,release_date,overview
0,tt0437086,Alita: Battle Angel,2019-01-31,When Alita awakens with no memory of who she i...
1,tt0448115,Shazam!,2019-03-23,A boy is given the ability to become an adult ...
2,tt0837563,Pet Sematary,2019-04-04,"Dr. Louis Creed and his wife, Rachel, move fro..."
3,tt0859635,Super Troopers 2,2018-04-19,When an international border dispute arises be...
4,tt10003978,Fiddler: A Miracle of Miracles,2019-07-18,The origin story behind one of Broadway's most...


In [67]:
type(df['release_date'][0])

str

In [61]:
DB_NAME = 'Movies'
create_query = """
      CREATE TABLE md_movies
    (
      imdb_id varchar(24),
      title varchar(50),
      release_date INT(4),
      overview varchar(50),
      FOREIGN KEY (imdb_id) REFERENCES imdb_movies2(imdb_id),
      ID INT AUTO_INCREMENT PRIMARY KEY
      )
    """
# cursor.execute(create_query)

In [53]:
def get_overview(all_info):
    overviews = []
    for movie in all_info:
        try:
            info = movie['movie_results'][0]
            overview = info['overview']
            overviews.append(overview)
        except:
            overview = 'Null'
            overviews.append(overview)
    return overviews
# get_overview(all_info)

In [46]:
def get_title(all_info):
    titles = []
    for movie in all_info:
        try:
            info = movie['movie_results'][0]
            title = info['original_title']
            titles.append(title)
        except:
            title = 'Null'
            titles.append(title)
    return titles
# get_title(all_info)

In [47]:
def get_release_date(all_info):
    release_dates = []
    for movie in all_info:
        try:
            info = movie['movie_results'][0]
            release_date = info['release_date']
            release_dates.append(release_date)
        except:
            release_date = 'Null'
            release_dates.append(release_date)
    return release_dates

In [72]:
def insert_movies(dataframe):
    all_movies_tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO Movies.md_movies (imdb_id_, title, release_date, overview) 
            VALUES (%s, %s, %s, %s);"""
    for movie in all_movies_tuples:
        
        cursor.execute(insert_statement, movie)
        print(movie)
    
#     cursor.executemany(insert_statement, all_movies_tuples)
    db.commit()

In [74]:
insert_movies(make_df(all_info))

('tt0437086', 'Alita: Battle Angel', '2019-01-31', 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.')
('tt0448115', 'Shazam!', '2019-03-23', 'A boy is given the ability to become an adult superhero in times of need with a single magic word.')
('tt0837563', 'Pet Sematary', '2019-04-04', 'Dr. Louis Creed and his wife, Rachel, move from Boston to Ludlow, in rural Maine, with their two young children. Hidden in the woods near the new family home, Ellie, their eldest daughter, discovers a mysterious cemetery where the pets of community members are buried.')
('tt0859635', 'Super Troopers 2', '2018-04-19', 'When an international border dispute arises between the U.S. and Canada, the Super Troopers- Mac, Thorny, Foster, Rabbit and Farva, are called in to set up a new Highway Patrol 

('tt10648550', 'Null', 'Null', 'Null')
('tt10678786', 'Null', 'Null', 'Null')
('tt1072748', 'Null', 'Null', 'Null')
('tt10777384', 'Null', 'Null', 'Null')
('tt10889642', 'Null', 'Null', 'Null')
('tt10954574', 'Null', 'Null', 'Null')
('tt1137450', 'Null', 'Null', 'Null')
('tt1206885', 'Null', 'Null', 'Null')
('tt1213641', 'Null', 'Null', 'Null')
('tt1226837', 'Null', 'Null', 'Null')
('tt1235187', 'Null', 'Null', 'Null')
('tt1255919', 'Null', 'Null', 'Null')
('tt1259528', 'Null', 'Null', 'Null')
('tt1270797', 'Null', 'Null', 'Null')
('tt1273221', 'Null', 'Null', 'Null')
('tt1285009', 'Null', 'Null', 'Null')
('tt1298644', 'Null', 'Null', 'Null')
('tt1308728', 'Null', 'Null', 'Null')
('tt1318517', 'Null', 'Null', 'Null')
('tt1365519', 'Null', 'Null', 'Null')
('tt1389098', 'Null', 'Null', 'Null')
('tt1413492', 'Null', 'Null', 'Null')
('tt1477834', 'Null', 'Null', 'Null')
('tt1478839', 'Null', 'Null', 'Null')
('tt1489887', 'Null', 'Null', 'Null')
('tt1502407', 'Null', 'Null', 'Null')
('tt151

('tt3009772', 'Ode to Joy', '2019-04-04', 'Charlie suffers from a serious disease that causes acute narcolepsy whenever he is happy. Although he develops a variety of techniques to deny himself pleasure and deal with his sleep issues, they’re put to the test when he falls in love.')
('tt3076510', 'Henchmen', '2018-12-07', 'In a world of super-villains, evil schemes and global domination… someone has to take out the Trash. Welcome to the world of Henchmen, third class. When a fresh-faced new recruit joins the Union of Evil, he is assigned to a motley crew of blue-collar workers led by fallen henchmen Hank. But when “The Kid” accidentally steals the super villain’s ultimate weapon, Hank must break his “risk nothing” code to save the boy he’s befriended, even if it means becoming the one thing he has always avoided… being a hero.')
('tt3104988', 'Crazy Rich Asians', '2018-08-15', "An American-born Chinese economics professor accompanies her boyfriend to Singapore for his best friend's wed

('tt4560436', 'Null', 'Null', 'Null')
('tt4566780', 'Null', 'Null', 'Null')
('tt4575576', 'Null', 'Null', 'Null')
('tt4595882', 'Null', 'Null', 'Null')
('tt4602066', 'Null', 'Null', 'Null')
('tt4614612', 'Null', 'Null', 'Null')
('tt4633694', 'Spider-Man: Into the Spider-Verse', '2018-12-06', 'Miles Morales is juggling his life between being a high school student and being a spider-man. When Wilson "Kingpin" Fisk uses a super collider, others from across the Spider-Verse are transported to this dimension.')
('tt4669264', 'Beirut', '2018-04-11', "In 1980s Beirut, Mason Skiles is a former U.S. diplomat who is called back into service to save a colleague from the group that is possibly responsible for his own family's death. Meanwhile, a CIA field agent who is working under cover at the American embassy is tasked with keeping Mason alive and ensuring that the mission is a success.")
('tt4669296', 'Lords of Chaos', '2019-02-08', "A teenager's quest to launch Norwegian Black Metal in Oslo in

('tt5092380', 'Dans la brume', '2018-04-04', 'Out of nowhere, a strange, deadly mist envelops Paris in minutes. Finding refuge on the highest floor of their apartment building, a couple attempts to figure out how to protect their bubble-bound daughter, whose capsule is left running on battery power. But as the hours go by, one thing becomes quite clear: the mist is closing in on them & if they want to survive, they will have to risk exposure at all costs.')
('tt5095030', 'Ant-Man and the Wasp', '2018-07-04', 'Just when his time under house arrest is about to end, Scott Lang once again puts his freedom at risk to help Hope van Dyne and Dr. Hank Pym dive into the quantum realm and try to accomplish, against time and any chance of success, a very dangerous rescue mission.')
('tt5104604', 'Isle of Dogs', '2018-03-23', "In the future, an outbreak of canine flu leads the mayor of a Japanese city to banish all dogs to an island that's a garbage dump. The outcasts must soon embark on an epic j

...

('tt7108976', 'Null', 'Null', 'Null')
('tt7112154', 'Null', 'Null', 'Null')
('tt7117594', 'Null', 'Null', 'Null')
('tt7125774', 'Null', 'Null', 'Null')
('tt7125860', 'Null', 'Null', 'Null')
('tt7131622', 'Null', 'Null', 'Null')
('tt7137380', 'Null', 'Null', 'Null')
('tt7137846', 'Null', 'Null', 'Null')
('tt7147540', 'Null', 'Null', 'Null')
('tt7153766', 'Null', 'Null', 'Null')
('tt7156436', 'Null', 'Null', 'Null')
('tt7158430', 'Null', 'Null', 'Null')
('tt7158582', 'Null', 'Null', 'Null')
('tt7163824', 'Null', 'Null', 'Null')
('tt7168728', 'Null', 'Null', 'Null')
('tt7204348', 'Null', 'Null', 'Null')
('tt7212726', 'Null', 'Null', 'Null')
('tt7218518', 'Null', 'Null', 'Null')
('tt7220754', 'Null', 'Null', 'Null')
('tt7231342', 'Null', 'Null', 'Null')
('tt7233726', 'Null', 'Null', 'Null')
('tt7236034', 'Null', 'Null', 'Null')
('tt7242142', 'Null', 'Null', 'Null')
('tt7250056', 'Null', 'Null', 'Null')
('tt7253216', 'Null', 'Null', 'Null')
('tt7255568', 'Null', 'Null', 'Null')
('tt7262882'

('tt7545566', 'Skate Kitchen', '2018-08-10', "Camille's life as a lonely suburban teenager changes dramatically when she befriends a group of girl skateboarders. As she journeys deeper into this raw New York City subculture, she begins to understand the true meaning of friendship as well as her inner self.")
('tt7547410', 'Dora and the Lost City of Gold', '2019-08-08', 'Dora, a girl who has spent most of her life exploring the jungle with her parents, now must navigate her most dangerous adventure yet: high school. Always the explorer, Dora quickly finds herself leading Boots (her best friend, a monkey), Diego, and a rag tag group of teens on an adventure to save her parents and solve the impossible mystery behind a lost Inca civilization.')
('tt7549776', 'Tod@s Caen', '2019-08-30', 'A pair of seducers try to prove who has the best techniques while trying not to fall in love with each other.')
('tt7555774', 'En guerre', '2018-05-16', 'After promising 1100 employees that they would prot

('tt8186318', 'Overcomer', '2019-08-22', 'After reluctantly agreeing to coach cross-country, high school basketball Coach John Harrison helps the least likely runner attempt the impossible in the biggest race of the year.')
('tt8223844', 'ちいさな英雄－カニとタマゴと透明人間－', '2018-08-24', 'From director Hiromasa Yonebayashi: “Kanini & Kanino”; “Life Ain’t Gonna Lose” directed by Yoshiyuki Momose; and “Invisible” directed by Akihiko Yamashita. These leaders of animation draw tales of modest heroes of our times.')
('tt8228538', 'Papi Chulo', '2019-06-07', 'A lonely TV weatherman strikes up an unusual friendship with a middle-aged Latino migrant worker.')
('tt8228884', 'I Got the Hook Up 2', '2019-07-12', "After best friends Black and Blue's restaurant is shut down, Black needs to find some cash -- fast. He thinks his luck has turned when Blue's son, Fatboy, and his best friend, Spyda,, bring him a stash of stolen cellphones, and Black decides to sell them on the streets. There's only one problem: The b

('tt8709508', 'Null', 'Null', 'Null')
('tt8723270', 'Null', 'Null', 'Null')
('tt8725958', 'Null', 'Null', 'Null')
('tt8726098', 'Null', 'Null', 'Null')
('tt8740778', 'Null', 'Null', 'Null')
('tt8747304', 'Null', 'Null', 'Null')
('tt8755316', 'Null', 'Null', 'Null')
('tt8760684', 'Null', 'Null', 'Null')
('tt8769032', 'Null', 'Null', 'Null')
('tt8769848', 'Null', 'Null', 'Null')
('tt8772262', 'Null', 'Null', 'Null')
('tt8820136', 'Null', 'Null', 'Null')
('tt8835552', 'Null', 'Null', 'Null')
('tt8844204', 'Null', 'Null', 'Null')
('tt8856090', 'Null', 'Null', 'Null')
('tt8869978', 'Null', 'Null', 'Null')
('tt8870902', 'Null', 'Null', 'Null')
('tt8908002', 'Null', 'Null', 'Null')
('tt8923482', 'Null', 'Null', 'Null')
('tt8925540', 'Null', 'Null', 'Null')
('tt8956390', 'Null', 'Null', 'Null')
('tt8973144', 'Null', 'Null', 'Null')
('tt8991268', 'Null', 'Null', 'Null')
('tt8999864', 'Null', 'Null', 'Null')
('tt9015306', 'Null', 'Null', 'Null')
('tt9019352', 'Null', 'Null', 'Null')
('tt9024106'

('tt9497146', '機動戦士ガンダムNT', '2018-11-30', 'U.C. 0097, one year after the opening of "Laplace\'s Box".  Despite the revelation of the Universal Century Charter that acknowledges the existence and rights of Newtypes, the framework of the world has not been greatly altered.  The conflict later dubbed the "Laplace Incident" is thought to have ended with the downfall of the Neo Zeon remnants known as the Sleeves. In its final battle, two full psycho-frame mobile suits displayed power beyond human understanding. The white unicorn and the black lion were sealed away to remove this danger from people\'s consciousness, and they should now be completely forgotten.  However, the RX-0 Unicorn Gundam 03, which disappeared two years earlier, is now about to show itself in the Earth Sphere once more. A golden phoenix… named Phenex.')
('tt9507276', '劇場版 幼女戦記', '2019-02-08', "The time is UC 1926. The Imperial Army's 203rd Air Mage Battalion led by Major Tanya Degurechaff has won the battle to the south